In [6]:
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.svm import SVC
from collections import Counter
from sklearn.metrics import accuracy_score
import joblib
from sqlalchemy import create_engine
from config import db_password

In [40]:
# read in interest rate csv file
interest = pd.read_csv("Clean_Data/clean_fedfunds.csv")
# change data types from int to string
interest["Month"] = interest["Month"].astype(str)
interest["Year"] = interest["Year"].astype(str)
# filter from 2002
interest = interest.loc[interest["Year"] >= "2002"]
interest.head()

,Year,Month,FEDFUNDS
570,2002,1,1.73
571,2002,2,1.74
572,2002,3,1.73
573,2002,4,1.75
574,2002,5,1.75


In [41]:
# read in industrial output csv file
indus = pd.read_csv("Clean_Data/Industrial.csv")
#change data types from int to string
indus["Month"] = indus["Month"].astype(str)
indus["Year"] = indus["Year"].astype(str)
indus.head()

,DATE,INDPRO,Month,Year
0,2002-01-01,88.6706,1,2002
1,2002-02-01,88.6723,2,2002
2,2002-03-01,89.3624,3,2002
3,2002-04-01,89.7933,4,2002
4,2002-05-01,90.1629,5,2002


In [42]:
# read in Retail Sales data 
sales = pd.read_csv("Clean_Data/Retail_Sales.csv")
# change out int to string
sales["Month"] = sales["Month"].astype(str)
sales["Year"] = sales["Year"].astype(str)
sales.head()

,DATE,RSXFS,Year,Month
0,2002-01-01,256307,2002,1
1,2002-02-01,257670,2002,2
2,2002-03-01,257059,2002,3
3,2002-04-01,261333,2002,4
4,2002-05-01,257573,2002,5


In [43]:
# read in inflation data
inflation = pd.read_csv("Clean_Data/inflation.csv")
#change to string
inflation["Year"] = inflation["Year"].astype(str)
inflation.head()

,Year,1,2,3,4,5,6,7,8,9,10,11,12
0,2002,2.6,2.6,2.4,2.5,2.5,2.3,2.2,2.4,2.2,2.2,2.0,1.9
1,2003,1.9,1.7,1.7,1.5,1.6,1.5,1.5,1.3,1.2,1.3,1.1,1.1
2,2004,1.1,1.2,1.6,1.8,1.7,1.9,1.8,1.7,2.0,2.0,2.2,2.2
3,2005,2.3,2.4,2.3,2.2,2.2,2.0,2.1,2.1,2.0,2.1,2.1,2.2
4,2006,2.1,2.1,2.1,2.3,2.4,2.6,2.7,2.8,2.9,2.7,2.6,2.6


In [44]:
# read in unemployment data
unemployment = pd.read_csv("Clean_Data/Unemployment.csv")
#change int to string
unemployment["Year"] = unemployment["Year"].astype(str)
unemployment.head()

,Year,1,2,3,4,5,6,7,8,9,10,11,12
0,2002,5.7,5.7,5.7,5.9,5.8,5.8,5.8,5.7,5.7,5.7,5.9,6.0
1,2003,5.8,5.9,5.9,6.0,6.1,6.3,6.2,6.1,6.1,6.0,5.8,5.7
2,2004,5.7,5.6,5.8,5.6,5.6,5.6,5.5,5.4,5.4,5.5,5.4,5.4
3,2005,5.3,5.4,5.2,5.2,5.1,5.0,5.0,4.9,5.0,5.0,5.0,4.9
4,2006,4.7,4.8,4.7,4.7,4.6,4.6,4.7,4.7,4.5,4.4,4.5,4.4


In [95]:
# create empty dataframe
data = pd.DataFrame()
ticker_list = ["AAPL", "NVDA", "AMZN", "TSLA", "MSFT", "GOOG", "META", "GOOGL", "PEP", "COST", "AVGO", "ADBE", "CSCO", "CMCSA", "TMUS", "INTC", "TXN", "AMD", "QCOM", "AMGN", "HON", "INTU", "FISV", "MRNA", "PANW", "KDP", "KLAC", "MNST", "SNPS", "AEP"]
#read in tickers
for i in ticker_list:
    temp = pd.read_csv(f"Extracting_Data/{i}/{i}_monthly_1_False.csv")
    # add ticker name to each row
    temp["Ticker"] = i
    # combines each ticker csv on top of eachother
    data = pd.concat([data, temp])
data.head()

,Datetime,Open,Close,High,Low,Volume,Gain%,Ticker
0,2002-06-30 22:00:00,0.316250,0.272500,0.335536,0.246429,8118415200,-13.833992,AAPL
1,2002-07-31 22:00:00,0.269821,0.263393,0.290179,0.249464,5151686400,-2.382529,AAPL
2,2002-08-31 22:00:00,0.258750,0.258929,0.271250,0.250893,5291837600,0.069014,AAPL
3,2002-09-30 22:00:00,0.260536,0.286964,0.293571,0.238571,6611432800,10.143926,AAPL
4,2002-10-31 22:00:00,0.284643,0.276786,0.310357,0.268036,4531721600,-2.760345,AAPL


In [96]:


#add a future column where shows the gain and add column to see where tickers overlap
data['future'] = data["Gain%"].shift(-1)
data['past'] = data["Gain%"].shift(1)
data["future_Ticker"] = data["Ticker"].shift(-1)
data["past_Ticker"] = data["Ticker"].shift(1)
data = data.dropna()

#drop rows where ticker info overlaps
data = data.loc[(data["future_Ticker"] == data["Ticker"]) & (data["past_Ticker"] == data["Ticker"])]
data.head()

,Datetime,Open,Close,High,Low,Volume,Gain%,Ticker,future,past,future_Ticker,past_Ticker
1,2002-07-31 22:00:00,0.269821,0.263393,0.290179,0.249464,5151686400,-2.382529,AAPL,0.069014,-13.833992,AAPL,AAPL
2,2002-08-31 22:00:00,0.258750,0.258929,0.271250,0.250893,5291837600,0.069014,AAPL,10.143926,-2.382529,AAPL,AAPL
3,2002-09-30 22:00:00,0.260536,0.286964,0.293571,0.238571,6611432800,10.143926,AAPL,-2.760345,0.069014,AAPL,AAPL
4,2002-10-31 22:00:00,0.284643,0.276786,0.310357,0.268036,4531721600,-2.760345,AAPL,-9.874217,10.143926,AAPL,AAPL
5,2002-11-30 22:00:00,0.283929,0.255893,0.287500,0.246071,4530439200,-9.874217,AAPL,0.000000,-2.760345,AAPL,AAPL


In [97]:
len(data)

6348

In [99]:
def data_transform(data):
    #create arrays
    inflation_stat = []
    unemployment_stat = []
    #iterate through each datetime in the dataset
    for i in data["Datetime"]:
        #calculates if the month is formatted as 0# and assigns month to number #
        if i[5] == "0":
            month = i[6]
        # if formatted as ## then month assigned this number ##
        else:
            month = i[5:7]

        # locates in inflation/unemployment data when the year is the same as the current itteration
        inf_month = inflation.loc[inflation["Year"] == [i][0][:4]]
        unemp_month = unemployment.loc[unemployment["Year"] == [i][0][:4]]
        
        #locates the month of the filtered data and extracts the value
        inflation_stat.append(inf_month[month].values[0])
        unemployment_stat.append(unemp_month[month].values[0])

    # adds arrays to dataset
    data["Inflation"] = inflation_stat
    data["Unemployment"] = unemployment_stat
    # eliminates month from data
    data = data.loc[data["Datetime"] < "2022-06"]
    
    # create arrays
    industrial_stat = []
    interest_stat = []
    sales_stat = []
    #iterate through each datetime in the dataset
    for i in data["Datetime"]:
        #calculates if the month is formatted as 0# and assigns month to number #
        if i[5] == "0":
            month = i[6]
        # if formatted as ## then month assigned this number ##
        else:
            month = i[5:6]
        
        # filters datasets with the year in the iteration and when the month of the dataset is = "month"
        row = interest.loc[(interest["Year"] == [i][0][:4]) & (interest["Month"] == month)]
        ind = indus.loc[(indus["Year"] == [i][0][:4]) & (indus["Month"] == month)]
        sal = sales.loc[(sales["Year"] == [i][0][:4]) & (sales["Month"] == month)]
        
        # append stats to arrays
        interest_stat.append(row["FEDFUNDS"].values[0])
        industrial_stat.append(ind["INDPRO"].values[0])
        sales_stat.append(sal["RSXFS"].values[0])
    # adds arrays to dataset
    data["Industrial"] = industrial_stat
    data["Interest"] = interest_stat
    data["Retail_Sales"] = sales_stat
    
    # add new columns with percent changed
    data["Interest_pct"] = data["Interest"].pct_change()
    data["Inflation_pct"] = data["Inflation"].pct_change()
    data["Unemployment_pct"] = data["Unemployment"].pct_change()
    data["Industrial_pct"] = data["Industrial"].pct_change()
    data["Volume_pct"] = data["Volume"].pct_change()
    data["Retail_Sales_pct"] = data["Retail_Sales"].pct_change() 
    data = data.dropna()

    # calculates if the future gain is positive or negative
    R_g = []
    for x in data["future"]:
        if x <= 0:
            R_g.append(0)
        if x > 0:
            R_g.append(1)
    #add column for calculation
    data["R/G"] = R_g
    
    return data


In [100]:
# call function
data = data_transform(data)
data.head()

,Datetime,Open,Close,High,Low,Volume,Gain%,Ticker,future,past,...,Industrial,Interest,Retail_Sales,Interest_pct,Inflation_pct,Unemployment_pct,Industrial_pct,Volume_pct,Retail_Sales_pct,R/G
2,2002-08-31 22:00:00,0.258750,0.258929,0.271250,0.250893,5291837600,0.069014,AAPL,10.143926,-2.382529,...,90.7850,1.74,265043,0.005780,0.090909,-0.017241,-0.001002,0.027205,0.008654,1
3,2002-09-30 22:00:00,0.260536,0.286964,0.293571,0.238571,6611432800,10.143926,AAPL,-2.760345,0.069014,...,90.8931,1.75,260626,0.005747,-0.083333,0.000000,0.001191,0.249364,-0.016665,0
4,2002-10-31 22:00:00,0.284643,0.276786,0.310357,0.268036,4531721600,-2.760345,AAPL,-9.874217,10.143926,...,88.6706,1.73,256307,-0.011429,0.000000,0.000000,-0.024452,-0.314563,-0.016572,0
5,2002-11-30 22:00:00,0.283929,0.255893,0.287500,0.246071,4530439200,-9.874217,AAPL,0.000000,-2.760345,...,88.6706,1.73,256307,0.000000,-0.090909,0.035088,0.000000,-0.000283,0.000000,0
6,2002-12-31 22:00:00,0.256429,0.256429,0.274643,0.242143,6045832800,0.000000,AAPL,4.163781,-9.874217,...,88.6706,1.73,256307,0.000000,-0.050000,0.016949,0.000000,0.334492,0.000000,1


In [120]:
#index to test/predict for
tester = pd.read_csv("Extracting_Data/QQQ/QQQ_monthly_1_False.csv")
#add a future column where shows the gain for next row
tester['future'] = tester["Gain%"].shift(-1)
tester['past'] = tester["Gain%"].shift(1)
tester = tester.dropna()
tester.head()

,Datetime,Open,Close,High,Low,Volume,Gain%,future,past
1,2002-07-31 22:00:00,23.74,23.49,26.21,21.30,2043810216,-1.053075,-10.147441,-8.163265
2,2002-08-31 22:00:00,23.06,20.72,24.35,20.49,1669047392,-10.147441,17.407939,-1.053075
3,2002-09-30 22:00:00,20.91,24.55,25.04,19.76,2129150120,17.407939,13.746410,-10.147441
4,2002-10-31 22:00:00,24.37,27.72,28.29,24.14,1669954692,13.746410,-14.250528,17.407939
5,2002-11-30 22:00:00,28.42,24.37,28.79,24.28,1397055816,-14.250528,-1.132686,13.746410


In [121]:
tester = data_transform(tester)
tester.tail()

,Datetime,Open,Close,High,Low,Volume,Gain%,future,past,Inflation,...,Industrial,Interest,Retail_Sales,Interest_pct,Inflation_pct,Unemployment_pct,Industrial_pct,Volume_pct,Retail_Sales_pct,R/G
234,2021-12-31 22:00:00,399.05,363.05,402.28,334.15,1848326612,-9.021426,-4.837692,-0.107964,5.5,...,99.4076,0.09,513269,0.000000,0.122449,-0.071429,0.000000,0.497130,0.000000,0
235,2022-01-31 22:00:00,364.43,346.80,370.10,318.26,1523185568,-4.837692,4.856110,-9.021426,6.0,...,102.5479,0.08,576000,-0.111111,0.090909,0.025641,0.031590,-0.175911,0.122219,1
236,2022-02-28 22:00:00,345.75,362.54,371.83,317.45,1686866041,4.856110,-13.660042,-4.837692,6.4,...,103.5578,0.08,581768,0.000000,0.066667,-0.050000,0.009848,0.107459,0.010014,0
237,2022-03-31 22:00:00,362.81,313.25,369.31,312.60,1501958749,-13.660042,-1.454464,4.856110,6.5,...,104.0733,0.20,587570,1.500000,0.015625,-0.052632,0.004978,-0.109616,0.009973,0
238,2022-04-30 22:00:00,312.83,308.28,330.29,280.21,1951162721,-1.454464,-8.252005,-13.660042,6.2,...,105.4868,0.33,590259,0.650000,-0.046154,0.000000,0.013582,0.299079,0.004576,0


In [105]:
### model for stock tickers with only Open, Close, Volume

X = tester[["Open", "Close", "Volume"]]
y = tester["R/G"]

#train test and split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=45)
# Standardize data set
scaler = StandardScaler()
# Fit X_train data
scaler.fit(X_train)
#Transform scaled data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=45)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=45)

# Fitting the model
rf_model = rf_model.fit(X_resampled, y_resampled)

# Evaluate the model
pred = rf_model.predict(X_test_scaled)

print(f"Model Accuracy: {accuracy_score(y_test, pred)}")

Model Accuracy: 0.5


In [116]:
X = tester[["Interest_pct","Unemployment_pct", "Industrial_pct", "Retail_Sales_pct","Inflation_pct", "Volume_pct", "Gain%", "past"]]
y = tester["R/G"]

#train test and split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=45)
# Standardize data set
scaler = StandardScaler()
# Fit X_train data
scaler.fit(X_train)
#Transform scaled data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=45)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=45)

# Fitting the model
rf_model = rf_model.fit(X_resampled, y_resampled)

# Evaluate the model
pred = rf_model.predict(X_test_scaled)

print(f"Model Accuracy: {accuracy_score(y_test, pred)}")

Model Accuracy: 0.65


In [107]:
# save model 
joblib.dump(rf_model, 'future_prediction.joblib')

['future_prediction.joblib']

In [108]:


X = data[["Interest_pct","Unemployment_pct", "Industrial_pct", "Retail_Sales_pct","Inflation_pct"]]
y = data["R/G"]

#columns from index to test
TestX = tester[["Interest_pct","Unemployment_pct", "Industrial_pct", "Retail_Sales_pct","Inflation_pct"]]
Testy = tester["R/G"]

#train test and split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=45)
# Standardize data set
scaler = StandardScaler()
# Fit X_train data
scaler.fit(X_train)
#Transform scaled data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=45)

# Fitting the model
rf_model = rf_model.fit(X_resampled, y_resampled)

# Evaluate the model
pred = rf_model.predict(X_test_scaled)

# fit tester sample data
scaled = scaler.fit_transform(TestX)
# Evaluate the model
y_pred = rf_model.predict(scaled)
# find accuracy of this sample data
accuracy_score(Testy, y_pred)

print(f"Model Accuracy: {accuracy_score(y_test, pred)} , QQQ Accuracy: {accuracy_score(Testy, y_pred)}")


Model Accuracy: 0.6855702583490864 , QQQ Accuracy: 0.7341772151898734


In [109]:
# save model 
joblib.dump(rf_model, 'similarity_prediction.joblib')

['similarity_prediction.joblib']

In [117]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, pred)

array([[12,  9],
       [12, 27]])

In [118]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.50      0.57      0.69      0.53      0.63      0.39        21
          1       0.75      0.69      0.57      0.72      0.63      0.40        39

avg / total       0.66      0.65      0.61      0.65      0.63      0.40        60



In [112]:

# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.2162385364576845, 'Unemployment_pct'),
 (0.2088936858002699, 'Industrial_pct'),
 (0.19563742199037812, 'Retail_Sales_pct'),
 (0.18967289853728989, 'Inflation_pct'),
 (0.18955745721437764, 'Interest_pct')]

In [113]:
#get rid of hours minutes seconds
date = []
for i in data["Datetime"]:
    date.append(i[0:10])
data["Datetime"] = date 

date = []
for i in tester["Datetime"]:
    date.append(i[0:10])
tester["Datetime"] = date 

In [114]:
#export to csv
data.to_csv("Tech_Stocks.csv")
tester.to_csv("QQQ.csv")

In [ ]:
#Exporting data into SQL Database
Tech_Stocks.csv.to_sql(name='Tech_Stocks', if_exists='replace',con=engine)